**This scripts shows how to Map your *Predict_proba* results to Original Classes to get a quick overview** **

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [3]:
import pandas as pd
import numpy as np

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


In [4]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [5]:
train.head()

In [6]:
test.head()

**Lable Encoding the Output Multi classes**

In [7]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [8]:
print (lbl_enc.classes_)
print (y)
print (set(y))

**Split the data into Train , Test**

In [9]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

**Since the Main data are 'TEXT' , so we need to vectorize them. Just using simple TFIDF for this case**

In [10]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

**Let fit a simple Logistic Regression and see the results**

In [11]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % metrics.log_loss(yvalid, predictions))

**The Log-loss results are not that great . It still have scope to improve. 
However our prime objective in this exercise is to just map the pred_proba percentages to original classes**

**Mapping pred_prob to Lable encoded values**

In [12]:
predictions[0:4]

In [13]:

clf.classes_
pred_To_Class_map = pd.DataFrame(predictions, columns=clf.classes_)
print (pred_To_Class_map.head(5))

**Mapping pred_proba to Original Classes for a more clearer outlook**

In [14]:
pred_To_Class_map = pd.DataFrame(predictions, columns=lbl_enc.classes_)
print (pred_To_Class_map.head(5))

** Lets consider the 1st TEXT **

** As you can see the classifier has predicted the 1st TEST-set TEXT values to be classified as 64% EAP , 7% HPL & 28% MWS**

** Since EAP has the highest probability percentage(64%), as such it is predciting it to belong to class EAP ( or class 0 in term of label encoding) as  you can see from the prediction values **

**Please vote if you find it useful**